In [1]:
import numpy
print(hasattr(numpy, 'rec'))

True


In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [2]:
MAIN_PATH = f'/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Nonos_synaptic_blockers'

In [3]:
data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)

NameError: name 'PAIRINGS_PATH' is not defined

In [4]:
PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE/')
data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
#te_unit_pairs = pd.DataFrame(data_te['biTE'])
#te_unit_pairs.drop_duplicates(subset=['source', 'target'])
data_te.keys()

NameError: name 'filename' is not defined

In [5]:
filename = 'ID1851_N0_DIV100_DATE20240530_1342_spontaneous_.raw.h5'
#MAIN_PATH_PROCESSED = f'/itet-stor/kvulic/neuronies/single_neurons/3_Student_Projects/Ferran/2_Processed_Data/1742_HeartShape/1742_DIV32_HeartShape/'

PROCESSED_DATA_PATH = os.path.join(MAIN_PATH, f'Sorters/')
#PROCESSED_DATA_PATH = MAIN_PATH_PROCESSED
data = np.load(os.path.join(PROCESSED_DATA_PATH, f'{filename[:-3]}_processed.pkl'), allow_pickle=True)
spikes_extremum = pd.DataFrame(data['SPIKEMAT_EXTREMUM'])
spikes = np.array([[int(row[0]), float(row[1]), float(row[2])] for row in data['SPIKEMAT']])
PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')
OUTPUT_PATH = os.path.join(MAIN_PATH,f'Latency_plots/Latency_plots_TE_new')
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

SORTER_PATH = PROCESSED_DATA_PATH
FULL_SORTER_PATH = os.path.join(SORTER_PATH,'Sorter_'+filename)


area = filename.split('_')[1]
import pickle

# Open the file in binary mode
sys.modules['numpy.rec'] = np.rec
#with open(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), 'rb') as f:
#    data_te = np.array(pickle.load(f))
data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
exp_duration = data_te['EXPERIMENT_DURATION']
te_unit_pairs = pd.DataFrame(data_te['mTE'])
pairings = te_unit_pairs

with open(os.path.join(FULL_SORTER_PATH, 'wf_folder_curated/sparsity.json'), 'r') as file:
    sorting_info = json.load(file)

unit_ids = sorting_info['unit_ids']

spikes, electrodes_pre_all, electrodes_post_all, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all = get_electrode_unit_info_te(data, pairings, area, unit_ids)

In [6]:
idx = 4
pre_extremum = pre_extremum_all[idx]
post_extremum = post_extremum_all[idx]
unit_pre = unit_pre_all[idx]
unit_post = unit_post_all[idx]

In [7]:
def get_latency_with_extremum_V2(spikes_extremum, unit_ids, unit_pre, unit_post):
    # Filter spikes for input and output electrodes
    input_spikes = spikes_extremum[spikes_extremum['UnitIdx'] == unit_ids.index(unit_pre)].reset_index(drop=True)
    output_spikes = spikes_extremum[spikes_extremum['UnitIdx'] == unit_ids.index(unit_post)].reset_index(drop=True)
    
    # Create a list to store latencies
    latencies = []
    input_spike_count = 0
    input_time = None
    
    # Combine all spikes and sort by time
    all_spikes = pd.concat([
        input_spikes.assign(is_input=True),
        output_spikes.assign(is_input=False)
    ]).sort_values('Spike_Time').reset_index(drop=True)
    
    for _, spike in all_spikes.iterrows():
        current_time = spike['Spike_Time']
        
        if spike['is_input']:
            # Record as input spike
            latencies.append((input_spike_count, current_time, 0, "input"))
            input_time = current_time
            input_spike_count += 1
        elif input_time is not None:  # This is an output spike and we have a previous input
            # Only record output if it's within 10ms window
            if current_time - input_time < 10:
                latencies.append((input_spike_count-1, current_time, current_time - input_time, "output"))
    
    # Convert list to structured numpy array
    if not latencies:  # If no latencies were found
        return np.array([], dtype=[('input spike', 'i4'), ('spike time', 'f8'),
                                 ('latency', 'f4'), ('category', 'U6')])
    
    return np.array(latencies, dtype=[('input spike', 'i4'), ('spike time', 'f8'),
                                    ('latency', 'f4'), ('category', 'U6')])

In [8]:
def get_latency_all(spikes, spikes_extremum, unit_ids, unit_pre, unit_post, input_electrode_number, output_electrode_number):
    # Filter spikes for input and output electrodes
    input_spikes = spikes_extremum[spikes_extremum['UnitIdx'] == unit_ids.index(unit_pre)].reset_index(drop=True)
    #input_spikes = pd.DataFrame(spikes[spikes[:, 0] == input_electrode_number], columns=['Electrode', 'Spike_Time', 'Amplitude']).reset_index(drop=True)
    #output_spikes = spikes[spikes['UnitIdx'] == unit_ids.index(unit_post)].reset_index(drop=True)
    output_spikes = pd.DataFrame(spikes[spikes[:, 0] == output_electrode_number], columns=['Electrode', 'Spike_Time', 'Amplitude']).reset_index(drop=True)

    # Create a list to store latencies
    latencies = []
    input_spike_count = 0
    input_time = None
    
    # Combine all spikes and sort by time
    all_spikes = pd.concat([
        input_spikes.assign(is_input=True),
        output_spikes.assign(is_input=False)
    ]).sort_values('Spike_Time').reset_index(drop=True)
    
    for _, spike in all_spikes.iterrows():
        current_time = spike['Spike_Time']
        
        if spike['is_input']:
            # Record as input spike
            latencies.append((input_spike_count, current_time, 0, "input"))
            input_time = current_time
            input_spike_count += 1
        elif input_time is not None:  # This is an output spike and we have a previous input
            # Only record output if it's within 10ms window
            if current_time - input_time < 10:
                latencies.append((input_spike_count-1, current_time, current_time - input_time, "output"))
    
    # Convert list to structured numpy array
    if not latencies:  # If no latencies were found
        return np.array([], dtype=[('input spike', 'i4'), ('spike time', 'f8'),
                                 ('latency', 'f4'), ('category', 'U6')])
    
    return np.array(latencies, dtype=[('input spike', 'i4'), ('spike time', 'f8'),
                                    ('latency', 'f4'), ('category', 'U6')])

In [9]:
def plot_color_coded_electrodes(save_path, unit_pre, unit_post, electrodes_pre, electrodes_post, filename, pre_extremum, post_extremum):
    
    electrode_values_pre = []
    color_values_pre = []

    electrode_values_post = []
    color_values_post = []

    colormap_pre = plt.get_cmap('Blues')
    colormap_post = plt.get_cmap('RdPu')
    num_elecs_pre = len(electrodes_pre)
    num_elecs_post = len(electrodes_post)
    colors_pre = [colormap_pre(i / num_elecs_pre) for i in range(num_elecs_pre)]
    colors_post = [colormap_post(i / num_elecs_post) for i in range(num_elecs_post)]
    
    electrode_values_pre.extend(electrodes_pre)
    color_values_pre.extend(colors_pre)
    electrode_values_post.extend(electrodes_post)
    color_values_post.extend(colors_post)

    color_values = []
    color_values.extend(colors_pre)
    color_values.extend(colors_post)
    elec_to_color = {'electrode': electrode_values_pre + electrode_values_post, 'color': color_values}

    plt.figure(figsize=(8, 5))
    plt.gca().invert_yaxis()
    for electrode in electrodes_pre:
        x, y = convert_elno_to_xy(electrode)
        plt.scatter(y, x, color=elec_to_color['color'][elec_to_color['electrode'].index(electrode)])
    for electrode in electrodes_post:
        x, y = convert_elno_to_xy(electrode)
        plt.scatter(y, x, color=elec_to_color['color'][elec_to_color['electrode'].index(electrode)])
    x,y = convert_elno_to_xy(pre_extremum[0])
    plt.scatter(y, x, color='yellow', marker='x')
    x,y = convert_elno_to_xy(post_extremum[0])
    plt.scatter(y, x, color='green', marker='x')
    plt.xlim(0, 220)
    plt.ylim(0, 120)
    plt.title(f"Pair presynaptic {unit_pre} and postsynaptic {unit_post}")
    plt.savefig(os.path.join(save_path, f"{filename[:-3]}_color_code_pre_{unit_pre}_post_{unit_post}.pdf"), format='pdf', dpi=300)
    plt.savefig(os.path.join(save_path, f"{filename[:-3]}_color_code_pre_{unit_pre}_post_{unit_post}.png"), format='png')
    plt.close()
    #plt.show()
    return elec_to_color

In [11]:
#input_electrodes = sorted(data['UNIT_TO_EL'][unit_pre])
#output_electrodes = sorted(data['UNIT_TO_EL'][unit_post])
#elec_to_color = plot_color_coded_electrodes('temp_path', unit_pre, unit_post, input_electrodes, output_electrodes, filename, pre_extremum, post_extremum)

In [ ]:
#make input and output electrodes one list sorted
#all_electrodes = input_electrodes + output_electrodes
#all_electrodes = sorted(all_electrodes)

In [38]:
def check_pair(latency, lag, exp_duration):
    num_input_spikes = latency[latency['category'] == 'input'].shape[0]
    num_rel_output_spikes = np.sum((latency['category'] == 'output') & (latency['latency'] >= lag - 1) & (latency['latency'] <= lag + 1))
    if num_input_spikes / exp_duration > 0.5:
        if num_rel_output_spikes / exp_duration > 0.2:
            print('Good', num_rel_output_spikes / exp_duration)
            return 'good'
        else:
            print('Bad',num_rel_output_spikes / exp_duration)
            return 'bad'
    else:
        return 'bad'

In [35]:
def plot_latency_and_location_with_extremum_here(save_path, filename, lag, exp_duration, input_ids, output_ids, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output=None):
    #include_input_in_latency = True
    #gain = 6.3
    for idx, input_id in enumerate(input_ids):
        #if include_input_in_latency:
        #    output_ids = list(set(output_ids + [id for id in input_ids if id != input_id]))
        latency_extremum = None
        input_electrode = np.array([input_id], dtype=float)
        #input_color = 'blue'
        input_color = elec_to_color['color'][elec_to_color['electrode'].index(input_id)]
        #input_spikes = spikes[spikes[:, 0] == input_electrode]
        input_spikes = spikes_extremum['Spike_Time'][spikes_extremum['UnitIdx'] == unit_ids.index(unit_pre)]
        #output_electrodes = spikes[np.isin(spikes[:, 0], output_ids)]
        output_electrodes_filtered = np.array(output_ids)

        fig1, ax1 = plt.subplots()
        ax1.set_ylabel("Experiment Time (s)")
        ax1.set_xlim((0., 10.))
        ax1.set_xlabel("Latency (ms)")
        for output_electrode in output_electrodes_filtered:
            
            #output_color = 'pink'
            output_color = elec_to_color['color'][elec_to_color['electrode'].index(output_electrode)]
            if output_color is not None:
                latency_all = get_latency_all(spikes, spikes_extremum, unit_ids, unit_pre, unit_post, input_id, output_electrode)
                
                latency_all = latency_all[latency_all['latency'] < 10]

                input_before = latency_all[latency_all['category'] == 'input']
                output_before = latency_all[latency_all['category'] == 'output']
                
                #print(latency_all)
                #with open(os.path.join(save_path, f"{filename[:-3]}_input_el_{input_id}_output_el_{int(output_electrode)}_lag_{lag}.pkl"), 'wb') as f:
                #    pickle.dump(latency, f)

                
                ax1.scatter(output_before['latency'], output_before['spike time'] / 1000, s=7, label='output', color=output_color)
                ax1.scatter(input_before['latency'], input_before['spike time'] / 1000, s=7, label='input', color=input_color)

                if extremum_output is not None:
                    latency_extremum = get_latency_with_extremum_V2(spikes_extremum, unit_ids, unit_pre, unit_post)
                    latency_extremum = latency_extremum[latency_extremum['latency'] < 10]
                    output_extremum = latency_extremum[latency_extremum['category'] == 'output']
                    ax1.scatter(output_extremum['latency'], output_extremum['spike time'] / 1000, s=10, label='output extremum', color='green')
                    validation = check_pair(latency_extremum, lag, exp_duration)
                #ax1.scatter(latency_extremum_input['latency'], latency_extremum_input['spike time'] / 1000, s=20, label='input extremum', color='blue')
                plt.title(f"Source {unit_pre} to Target {unit_post}")
                ax1.legend()

            if validation == 'good':	
                OUTPUT_PATH = os.path.join(save_path, 'good_pairs')
                plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_input_el_{input_id}_unit_pre_{unit_pre}_output_el_{int(output_electrode)}_unit_post_{unit_post}_lag_{lag}_STTRP.pdf"), format='pdf')
                plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_input_el_{input_id}_unit_pre_{unit_pre}_output_el_{int(output_electrode)}_unit_post_{unit_post}_lag_{lag}_STTRP.png"), format='png')
                plt.close()
            elif validation == 'bad':
                OUTPUT_PATH = os.path.join(save_path, 'bad_pairs')
                plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_input_el_{input_id}_unit_pre_{unit_pre}_output_el_{int(output_electrode)}_unit_post_{unit_post}_lag_{lag}_STTRP.pdf"), format='pdf')
                plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_input_el_{input_id}_unit_pre_{unit_pre}_output_el_{int(output_electrode)}_unit_post_{unit_post}_lag_{lag}_STTRP.png"), format='png')
                plt.close()
        #plt.show()
            #plt.close()

    return latency_extremum
    #return latency_extremum

In [ ]:
def plot_latency_and_location_with_extremum_both_plots(save_path, filename, lag, exp_duration, unit_to_el, input_ids, output_ids, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output=None):
    # Create color mapping for electrodes
    input_electrodes = sorted(unit_to_el[unit_pre])
    output_electrodes = sorted(unit_to_el[unit_post])   
    electrodes_pre = input_electrodes
    electrodes_post = output_electrodes
    
    colormap_pre = plt.get_cmap('Blues')
    colormap_post = plt.get_cmap('RdPu')
    num_elecs_pre = len(electrodes_pre)
    num_elecs_post = len(electrodes_post)
    colors_pre = [colormap_pre(i / num_elecs_pre) for i in range(num_elecs_pre)]
    colors_post = [colormap_post(i / num_elecs_post) for i in range(num_elecs_post)]
    
    electrode_values_pre = electrodes_pre
    electrode_values_post = electrodes_post
    color_values = []
    color_values.extend(colors_pre)
    color_values.extend(colors_post)
    elec_to_color = {'electrode': electrode_values_pre + electrode_values_post, 'color': color_values}
    
    for idx, input_id in enumerate(input_ids):
        latency_extremum = None
        input_electrode = np.array([input_id], dtype=float)
        input_color = elec_to_color['color'][elec_to_color['electrode'].index(input_id)]
        #input_spikes = spikes_extremum['Spike_Time'][spikes_extremum['UnitIdx'] == unit_ids.index(unit_pre)]
        output_electrodes_filtered = np.array(output_ids)

        # Create figure with 2 subplots side by side
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # First subplot - Latency plot
        ax1.set_ylabel("Experiment Time (s)")
        ax1.set_xlim((0., 10.))
        ax1.set_xlabel("Latency (ms)")
        validation = None
        
        for output_electrode in output_electrodes_filtered:
            output_color = elec_to_color['color'][elec_to_color['electrode'].index(output_electrode)]
            if output_color is not None:
                latency_all = get_latency_all(spikes, spikes_extremum, unit_ids, unit_pre, unit_post, input_id, output_electrode)
                
                latency_all = latency_all[latency_all['latency'] < 10]

                input_before = latency_all[latency_all['category'] == 'input']
                output_before = latency_all[latency_all['category'] == 'output']
                
                ax1.scatter(output_before['latency'], output_before['spike time'] / 1000, s=7, label='output', color=output_color)
                ax1.scatter(input_before['latency'], input_before['spike time'] / 1000, s=7, label='input', color=input_color)

                if extremum_output is not None:
                    latency_extremum = get_latency_with_extremum_V2(spikes_extremum, unit_ids, unit_pre, unit_post)
                    latency_extremum = latency_extremum[latency_extremum['latency'] < 10]
                    output_extremum = latency_extremum[latency_extremum['category'] == 'output']
                    ax1.scatter(output_extremum['latency'], output_extremum['spike time'] / 1000, s=10, label='output extremum', color='green')
                    validation = check_pair(latency_extremum, lag, exp_duration)
                
                ax1.set_title(f"Source {unit_pre} to Target {unit_post}")
                handles, labels = ax1.get_legend_handles_labels()
                by_label = dict(zip(labels, handles))
                ax1.legend(by_label.values(), by_label.keys())
        
        # Second subplot - Electrode location plot
        ax2.invert_yaxis()
        for electrode in electrodes_pre:
            x, y = convert_elno_to_xy(electrode)
            ax2.scatter(y, x, color=elec_to_color['color'][elec_to_color['electrode'].index(electrode)])
        for electrode in electrodes_post:
            x, y = convert_elno_to_xy(electrode)
            ax2.scatter(y, x, color=elec_to_color['color'][elec_to_color['electrode'].index(electrode)])
        
        # Mark extremum points
        pre_extremum_electrode = input_ids[0] if len(input_ids) == 1 else input_ids
        post_extremum_electrode = output_ids[0] if len(output_ids) == 1 else output_ids
        
        x, y = convert_elno_to_xy(pre_extremum_electrode)
        ax2.scatter(y, x, color='yellow', marker='x', s=100)
        
        x, y = convert_elno_to_xy(post_extremum_electrode)
        ax2.scatter(y, x, color='green', marker='x', s=100)
        
        ax2.set_xlim(0, 220)
        ax2.set_ylim(0, 120)
        ax2.set_title(f"Pair presynaptic {unit_pre} and postsynaptic {unit_post}")
        
        plt.tight_layout()
        
        # Save figures based on validation
        if validation == 'good':
            OUTPUT_PATH = os.path.join(save_path, 'good_pairs')
            if os.path.exists(OUTPUT_PATH) == False:
                os.makedirs(OUTPUT_PATH)
            plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_pre_{unit_pre}_post_{unit_post}_lag_{lag}_combined.pdf"), format='pdf', dpi=300)
            plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_pre_{unit_pre}_post_{unit_post}_lag_{lag}_combined.png"), format='png')
            plt.close()
        elif validation == 'bad':
            if os.path.exists(OUTPUT_PATH) == False:
                os.makedirs(OUTPUT_PATH)
            OUTPUT_PATH = os.path.join(save_path, 'bad_pairs')
            plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_pre_{unit_pre}_post_{unit_post}_lag_{lag}_combined.pdf"), format='pdf', dpi=300)
            plt.savefig(os.path.join(OUTPUT_PATH, f"{filename[:-3]}_pre_{unit_pre}_post_{unit_post}_lag_{lag}_combined.png"), format='png')
            plt.close()
        elif validation is None:
            # If no validation is performed, save in the main path
            plt.savefig(os.path.join(save_path, f"{filename[:-3]}_pre_{unit_pre}_post_{unit_post}_lag_{lag}_combined.pdf"), format='pdf', dpi=300)
            plt.savefig(os.path.join(save_path, f"{filename[:-3]}_pre_{unit_pre}_post_{unit_post}_lag_{lag}_combined.png"), format='png')
            plt.close()

    return latency_extremum

In [41]:
for idx in range(len(pre_extremum_all)):
    pre_extremum = pre_extremum_all[idx]
    post_extremum = post_extremum_all[idx]
    unit_pre = unit_pre_all[idx]
    unit_post = unit_post_all[idx]
    lag = lag_all[idx]
    unit_to_el = data_te['UNIT_TO_EL']
    #nput_electrodes = sorted(data['UNIT_TO_EL'][unit_pre])
    #output_electrodes = sorted(data['UNIT_TO_EL'][unit_post])
    #elec_to_color = plot_color_coded_electrodes(OUTPUT_PATH, unit_pre, unit_post, input_electrodes, output_electrodes, filename, pre_extremum, post_extremum)
    latency = plot_latency_and_location_with_extremum_both_plots(OUTPUT_PATH, filename, lag, exp_duration, unit_to_el, pre_extremum, post_extremum, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output = post_extremum)
    #print(f"Lag {pairings['lag'][idx]}, TE {pairings['te'][idx]}")


Good 0.2798134576948701


2025-03-18 17:23:30,618 Assigning font /F1 = '/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'
2025-03-18 17:23:30,728 Embedding font /home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
2025-03-18 17:23:30,728 Writing TrueType font.
2025-03-18 17:23:31,401 Assigning font /F1 = '/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'
2025-03-18 17:23:31,508 Embedding font /home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
2025-03-18 17:23:31,509 Writing TrueType font.
2025-03-18 17:23:32,255 Assigning font /F1 = '/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'
2025-03-18 17:23:32,361 Embedding font /home/kvuli

Bad 0.039973351099267154


2025-03-18 17:23:33,359 Embedding font /home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
2025-03-18 17:23:33,359 Writing TrueType font.
2025-03-18 17:23:34,034 Assigning font /F1 = '/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'


Bad 0.02664890073284477


...

2025-03-18 17:24:42,700 Embedding font /home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
2025-03-18 17:24:42,701 Writing TrueType font.
2025-03-18 17:24:43,378 Assigning font /F1 = '/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'


Bad 0.013324450366422385


2025-03-18 17:24:43,486 Embedding font /home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
2025-03-18 17:24:43,486 Writing TrueType font.
2025-03-18 17:24:44,151 Assigning font /F1 = '/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'


Bad 0.039973351099267154


2025-03-18 17:24:44,259 Embedding font /home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/.venv/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
2025-03-18 17:24:44,259 Writing TrueType font.
